In this notebook, we will:
* get some information about the geometries in the data using some attributes and methods of the GeoSeries
* learn how to create a street map layer using folium

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

We can think of the `GeoSeries` as the geometry column of the geodataframe.

In [40]:
school_dists = gpd.read_file("assets/school-district.geojson")
type(school_dists.geometry)

geopandas.geoseries.GeoSeries

In [60]:
school_dists.head()

,first_name,city,zip,email,state,last_name,address,position,term_expir,district,phone,geometry,center
0,Dr. Sharon,Nashville,37218,gentryfordistrict1@comcast.net,TN,Gentry,6108 Beals Lane,Member,2016,1,615-268-5269,"MULTIPOLYGON (((-86.77136 36.38357, -86.77134 ...",POINT (-86.86087 36.26282)
1,Jill,Madison,37115,jill.speering@mnps.org,TN,Speering,1033 Falls Avenue,Vice-Chair,2016,3,615-562-5234,"MULTIPOLYGON (((-86.75365 36.40428, -86.75353 ...",POINT (-86.72361 36.28516)
2,Dr. Jo Ann,Nashville,37220,joann.brannon@mnps.org,TN,Brannon,5444 San Marcos Drive,Member,2018,2,615-833-5976,"MULTIPOLYGON (((-86.76696 36.08333, -86.76590 ...",POINT (-86.70156 36.03021)
3,Anna,Hermitage,37076,anna.shepherd@mnps.org,TN,Shepherd,4545 Raccoon Trail,Chair,2018,4,615-210-3768,"MULTIPOLYGON (((-86.58098 36.20935, -86.58099 ...",POINT (-86.63964 36.19697)
4,Amy,Nashville,37221,amy.frogge@mnps.org,TN,Frogge,7237 Riverfront Drive,Member,2016,9,615-521-5650,"MULTIPOLYGON (((-86.97287 36.20828, -86.97045 ...",POINT (-86.95428 36.10392)


The `GeoSeries` inherits a number of useful methods and attributes from the `shapely` package. Some of them are:

* `GeoSeries.area` : the are of each gemoetry in a `GeoSeries`.
* `GeoSeries.centroid` : the center point of each gemoetry in a `GeoSeries`.
* `GeoSeries.distance(other)` : the minimum distance to `other`.

### `GeoSeries.area`

In [41]:
school_dists.geometry.area

C:\Users\U\AppData\Local\Temp/ipykernel_45896/1901776299.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  school_dists.geometry.area


0    0.036641
1    0.014205
2    0.008328
3    0.014123
4    0.023030
5    0.010704
6    0.006415
7    0.007813
8    0.015004
dtype: float64

These values are in units equivalent to the sqaured of the units used for the distance in the goedataframe which depends on the CRS set for it.

In [42]:
school_dists.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Here, we use the CRS `epsg: 4326` which used degrees for distances thus degree-squared for areas. We can change the CRS to `epsg: 3857` to use meters instead of degrees for distance thus meter-squared. Changing the CRS can be done using `to_crs` method.

In [43]:
school_dists_3857 = school_dists.to_crs(epsg=3857)
school_dists_3857.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [46]:
# this will show the area in km^2
school_dists_3857.geometry.area / (10**3 * 10**3)

0    563.134380
1    218.369949
2    127.615396
3    216.871511
4    353.232132
5    164.137548
6     98.469632
7    119.742279
8    230.135653
dtype: float64

In [47]:
districts_areas_km2 = school_dists_3857.geometry.area / (10**3 * 10**3)
districts_areas_km2.sort_values()

6     98.469632
7    119.742279
2    127.615396
5    164.137548
3    216.871511
1    218.369949
8    230.135653
4    353.232132
0    563.134380
dtype: float64

### `GeoSeries.centroid`

This will return the center point of each geometry in the geometry geoseries.

In [48]:
school_dists.geometry.centroid

C:\Users\U\AppData\Local\Temp/ipykernel_45896/4168532340.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  school_dists.geometry.centroid


0    POINT (-86.86087 36.26282)
1    POINT (-86.72361 36.28516)
2    POINT (-86.70156 36.03021)
3    POINT (-86.63964 36.19697)
4    POINT (-86.95428 36.10392)
5    POINT (-86.82739 36.08342)
6    POINT (-86.75215 36.16995)
7    POINT (-86.63366 36.04946)
8    POINT (-86.64296 36.10810)
dtype: geometry

In [50]:
school_dists['center'] = school_dists.geometry.centroid

school_dists_centers = school_dists[['district', 'center']]

school_dists_centers.head()

C:\Users\U\AppData\Local\Temp/ipykernel_45896/881221759.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  school_dists['center'] = school_dists.geometry.centroid


,district,center
0,1,POINT (-86.86087 36.26282)
1,3,POINT (-86.72361 36.28516)
2,2,POINT (-86.70156 36.03021)
3,4,POINT (-86.63964 36.19697)
4,9,POINT (-86.95428 36.10392)


### `GeoSeries.distance(other)`

Let's say we want to know which school is the closest to the center of the school district number 1.

In [61]:
from shapely.geometry import Point

schools = pd.read_csv("assets/schools.csv")

schools["geometry"] = schools.apply(lambda row: Point((row.Longitude, row.Latitude)), axis = 1)

schools_gpd = gpd.GeoDataFrame(schools[["School Name", "geometry"]], geometry = "geometry", crs = "epsg:4326")

C:\Users\U\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [62]:
schools_gpd.head(3)

,School Name,geometry
0,A. Z. Kelley Elementary,POINT (-86.65885 36.02182)
1,Alex Green Elementary,POINT (-86.83223 36.25296)
2,Amqui Elementary,POINT (-86.70383 36.27377)


In [63]:
district_1 = school_dists.loc[school_dists.district == '1']
district_1

,first_name,city,zip,email,state,last_name,address,position,term_expir,district,phone,geometry,center
0,Dr. Sharon,Nashville,37218,gentryfordistrict1@comcast.net,TN,Gentry,6108 Beals Lane,Member,2016,1,615-268-5269,"MULTIPOLYGON (((-86.77136 36.38357, -86.77134 ...",POINT (-86.86087 36.26282)


In [65]:
schools_district_1 = gpd.sjoin(
    schools_gpd,
    district_1,
    predicate = "within"
) # this will return only the schools completely within the district 1.

schools_district_1.shape

(30, 15)

In [66]:
schools_district_1.head(1)

,School Name,geometry,index_right,first_name,city,zip,email,state,last_name,address,position,term_expir,district,phone,center
1,Alex Green Elementary,POINT (-86.83223 36.25296),0,Dr. Sharon,Nashville,37218,gentryfordistrict1@comcast.net,TN,Gentry,6108 Beals Lane,Member,2016,1,615-268-5269,POINT (-86.86087 36.26282)


In [69]:
import pprint # just for pretty printing...

# Create a dictionary of each school and its distance to the center of the district.
distances = dict()

for index, row in schools_district_1.iterrows():
    school_name = row["School Name"]
    center = row["center"]
    distance = row.geometry.distance(center)
    distances[school_name] = distance

pprint.pprint(distances)

{'Alex Green Elementary': 0.030287172719682773,
 'Bellshire Elementary': 0.0988045140909651,
 'Brick Church College Prep': 0.08961013862715599,
 'Buena Vista Elementary': 0.10570511270825833,
 'Cockrill Elementary': 0.1077685612196105,
 'Creswell Middle Prep School of the Arts': 0.07071695528308394,
 'Cumberland Elementary': 0.05107245423888074,
 'Haynes Middle': 0.09051735342452247,
 'Hull-Jackson Elementary': 0.09021711911422044,
 'Ivanetta H. Davis Learning Center at Bordeaux': 0.07662457433119775,
 'Joelton Elementary': 0.053341968112843634,
 'Joelton Middle': 0.05125961450823257,
 'John Early Middle': 0.09422796012374718,
 'Jones Elementary': 0.10102567625424698,
 'KIPP Nashville College Prep': 0.06251427354823408,
 'KIPP Nashville College Prep Elementary': 0.062254750966657696,
 'Moses McKissack Middle': 0.10691030507945888,
 'Nashville Academy of Computer Science': 0.08320287768739988,
 'Nashville Prep': 0.10066854450222189,
 'Purpose Prep': 0.0952621018938427,
 'RePublic High S

### Street maps with folium

We can add some a street map to our geospatial visualization using the `Map` class from the `folium` package.

```python
place = folium.Map(
    location = [latitude, longitude],
    zoom_start = 12
)
```

Here, we pass an array (list) of the center point of the map. It's important to notice that latitude is passed first then the longitude unlike the geometry point in our geodataframes.

In [81]:
import folium

eiffel_tower = folium.Map(location = [48.8583736, 2.2922926], zoom_start = 20) # [latitude, longitude]

display(eiffel_tower)

We want to extract a folium point from our geodataframe. So, we need to reverse the longitude and latitude of the center point. `Point` has `x` and `y` as its members, and we will use them to create the point we will pass to folium.

In [85]:
center_point = district_1.center[0]

district_center = [center_point.y, center_point.x]

print(center_point)
print(district_center)

POINT (-86.86086595994405 36.2628221811899)
[36.262822181189904, -86.86086595994405]


Now, we can generate the map of this district.

In [91]:
district_1_map = folium.Map(
    location = district_center,
    zoom_start = 12
)

display(district_1_map)

We can add the ploygon of the district by calling the `GeoJson` class from `folium`. Then, we add it to the map using `add_to(map)`.

In [92]:
folium.GeoJson(district_1.geometry).add_to(district_1_map)

display(district_1_map)

### Creating popups and markers in `folium`

We can add a marker using the `Marker` class from `folium`.

```python
marker = folium.Marker(location = location)

marker.add_to(district)
```

In [93]:
for index, row in schools_district_1.iterrows():
    location = [row.geometry.y, row.geometry.x]
    marker = folium.Marker(location = location)
    marker.add_to(district_1_map)

display(district_1_map)

To show a pop up when clicking on a marker, we add to the `Marker` class a popup message using the `popup` argument.

In [95]:
for index, row in schools_district_1.iterrows():
    location = [row.geometry.y, row.geometry.x]
    message = f'<strong> {row["School Name"]} </strong>'
    marker = folium.Marker(location = location, popup = message)
    marker.add_to(district_1_map)

display(district_1_map)

Now, when you click on a marker, you will see a popup message showing the school name.